# Hauser & Schwarz - analiza danych

Wracamy do eksperymentu I z artykułu "How seemingly innocuous words can bias judgment: Semantic prosody and impression formation". Będziemy chcieli powtórzyć analizę, którą wykonali autorzy. Mamy już kod do podstawowej obróbki danych, teraz musimy przeprowadzić odpowiednie analizy statystyczne na "wyczyszczonych" danych.

In [1]:
import pandas as pd
pd.set_option('max_columns', 50)
import re
import scipy.stats as stats
import statsmodels.api as sm
import pingouin as pg
import numpy as np

Autorzy w swojej analizie obliczyli współczynnik $\alpha$ Cronbacha. Nie znalazłem odpowiedniej funkcji w żadnej z dostępnych bibliotek, ale nie jest trudno ją zaimplementować. Tę implementację znalazłem w internecie, ale zwraca takie same wyniki jak odpowiednia funkcja w R.

In [2]:
def CronbachAlpha(itemscores):
    itemscores = np.asarray(itemscores)
    itemvars = itemscores.var(axis=0, ddof=1)
    tscores = itemscores.sum(axis=1)
    nitems = itemscores.shape[1]
    calpha = nitems / float(nitems-1) * (1 - itemvars.sum() / float(tscores.var(ddof=1)))

    return calpha

Wykonajmy kod, który napisaliśmy w poprzednim tygodniu:

In [3]:
data = pd.read_csv('semantic_prosody_utterly_totally_Daniel.csv')
totally = data.dropna(subset = ['tval']).dropna('columns')
utterly = data.dropna(subset = ['uval']).dropna('columns')
totally.columns = [col[2:] if re.search(string=col, pattern='^._') else col for col in totally.columns]
utterly.columns = [col[2:] if re.search(string=col, pattern='^._') else col for col in utterly.columns]
totally['adverb'] = 'totally'
utterly['adverb'] = 'utterly'
totally['val'] = totally['tval']
utterly['val'] = utterly['uval']
data = pd.concat([totally, utterly], sort=False)

Pierwszą rzeczą, którą zrobili autorzy (o której nie ma nic w artykule) było przeprowadzenie analizy czynnikowej. Ich hipotezą było to, że uda się za jej pomocą wyodrębnić dwa latentne czynniki - jeden dotyczący oceny "ciepła" osoby, drugi jej kompetencji. Dla zainteresowanych analizą czynnikową poniżej znajduje się kod wykonujący ją dla zebranych przez Hausera i Schwarza danych. Używa on funkcji `Factor` z modułu `statsmodels.multivariate`.

In [4]:
variables = ['warm',
        'gn',
        'sincere',
        'friendly',
        'wi',
        'trustworthy',
        'competent',
        'confident',
        'intel',
        'capable',
        'efficient']

In [5]:
model = sm.multivariate.Factor(data[variables], n_factor=10).fit()
model.rotate('varimax')
model.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                       Factor analysis results
=====================================================================================================
                              Eigenvalues                                                            
-----------------------------------------------------------------------------------------------------
        warm    gn   sincere friendly   wi   trustworthy competent confident intel  capable efficient
-----------------------------------------------------------------------------------------------------
       7.5449 0.8797  0.1664   0.0970 0.0789      0.0624    0.0514    0.0370 0.0249  0.0114   -0.0002
-----------------------------------------------------------------------------------------------------
                                                                                                     
-----------------------------------------------------------------------------------------------------
                              Communality                                                            
-----------------------------------------------------------------------------------------------------
        warm    gn   sincere friendly   wi   trustworthy competent confident intel  capable efficient
-----------------------------------------------------------------------------------------------------
       0.9239 0.8691  0.8243   0.8753 0.8543      0.8287    0.7992    0.7256 0.6846  0.8752    0.6939
-----------------------------------------------------------------------------------------------------
                                                                                                     
-----------------------------------------------------------------------------------------------------
                           Pre-rotated loadings                                                      
-----------------------------------------------------------------------------------------------------
            factor 0 factor 1 factor 2 factor 3 factor 4 factor 5 factor 6 factor 7 factor 8 factor 9
-----------------------------------------------------------------------------------------------------
warm          0.8621   0.3533   0.1920  -0.0963  -0.0013   0.0235  -0.0582   0.0176   0.0681   0.0272
gn            0.8781   0.2421   0.0842   0.0268   0.1317  -0.0008  -0.0838   0.0165  -0.0703  -0.0444
sincere       0.8501   0.1699  -0.2379  -0.0695  -0.0342   0.0187   0.0322   0.0810   0.0154  -0.0444
friendly      0.8636   0.2864   0.1041  -0.0493  -0.0860  -0.0812   0.1374  -0.0285  -0.0215  -0.0078
wi            0.8679   0.1880  -0.2211  -0.0143   0.0202  -0.0295  -0.0652  -0.0971   0.0280   0.0309
trustworthy   0.8580   0.1789  -0.0066   0.2241  -0.0226   0.0770   0.0465  -0.0302  -0.0230   0.0170
competent     0.7939  -0.3637   0.0697   0.0110  -0.1136   0.1110  -0.0390  -0.0377   0.0405  -0.0431
confident     0.8091  -0.1949  -0.0113   0.0309  -0.1066  -0.0377  -0.0622   0.1045  -0.0443   0.0474
intel         0.7266  -0.3409  -0.0062  -0.1579   0.0359   0.0654   0.0301  -0.0511  -0.0771   0.0220
capable       0.8222  -0.4052   0.0320   0.0518   0.0215  -0.1695  -0.0179  -0.0276   0.0253  -0.0207
efficient     0.7641  -0.2596   0.0033   0.0260   0.1613   0.0372   0.0898   0.0542   0.0560   0.0203
-----------------------------------------------------------------------------------------------------
                                                                                                     
-----------------------------------------------------------------------------------------------------
                           varimax rotated loadings                                                  
-----------------------------------------------------------------------------------------------------
            factor 0 factor 1 factor 2 factor 3 factor 4 factor 5 factor 6 factor 7 factor 8 factor 9
------------

Autorzy skonstruowali dwie skale - "ciepła" oraz "kompetencji" - i obliczyli współczynnik $\alpha$ Cronbacha aby ocenić ich wewnętrzną spójność. My również to zrobimy:

In [6]:
CronbachAlpha(data[['warm', 'gn', 'sincere', 'friendly', 'wi', 'trustworthy']])

0.9574364180654292

In [7]:
CronbachAlpha(data[['competent', 'confident', 'intel', 'capable', 'efficient', 'skillful']])

0.935411415916262

Autorzy do celów analiz skonstruowali dwie dodatkowe zmienne - wskaźnik "ciepła" oraz "kompetencji". Oba są po prostu średnimi z odpowiedzi na odpowiednie pytania.

In [8]:
# mean(1) oznacza, że wyciągamy średnią w wierszach tzn. dal każdego uczestnika
data['competenceind'] = data[['competent', 'confident', 'intel', 'capable', 'efficient', 'skillful']].mean(1)

In [9]:
data['warmind'] = data[['warm', 'gn', 'sincere', 'friendly', 'wi', 'trustworthy']].mean(1)

Następnie aby móc porównywać wartości trzech zmiennych (`competenceind`, `warmind` oraz `val`) autorzy je wystandaryzowali (obliczyli z-score dla każdej obserwacji). Powodem było to, że były na innych skalach (jedne na 5 punktowej, inne na 7 punktowej skali Likerta). My zrobimy to używając funkcji `zscore` z modułu `scipy.stats`.

In [10]:
data['zvalence'] = stats.zscore(data['val'])

In [11]:
data['zcompetenceind'] = stats.zscore(data['competenceind'])

In [12]:
data['zwarmind'] = stats.zscore(data['warmind'])

Teraz nasze dane należy sprowadzić do postaci "długiej". Jako zmiennej identyfikującej użyjemy `V1` - tam jest unikatowy identyfikator użytkownika panelu. Zmienne z wartościami to `zwarmind`, `zcompetenceind` oraz `zvalence`. 

In [13]:
data_l = pd.melt(data, 
                 id_vars= ['adverb', 'V1'], 
                 value_vars = ['zwarmind', 'zcompetenceind', 'zvalence'],
                var_name = 'rating')

Wreszcie możemy przeprowadzić tak jak w artykule mieszaną analizę wariancji. Do zrobienia tego użyjemy funkcji `mixed_anova` z pakietu `pingouin`. Naszą zmienną wewnątrzgrupową jest `rating` (bo mamy 3 rodzaje ratingu - `zwarmind`, `zcompetenceind` i `zvalence`), a zmienną międzygrupową jest `adverb` (bo każdy badany dostawał tylko jedną z dwóch różniących się przysłówkiem wersji historyjki). Musimy również powiedzieć która kolumna identyfikuje badanego (u nas jest to `V1`).

In [14]:
pg.mixed_anova(data = data_l, between = 'adverb', within = 'rating', subject = 'V1', dv = 'value')

,Source,SS,DF1,DF2,MS,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
0,adverb,17.845,1,561,17.845,7.543,0.006217,-,0.013,-,-,-,-
1,rating,0.000,2,1122,0.000,0.000,1.000000,1,0.000,0.89,False,0.876,6.95815e-17
2,Interaction,1.664,2,1122,0.832,2.728,0.065815,-,0.005,-,-,-,-


Łał! Udało nam się otrzymać dokładnie taką samą analizę jak ta wykonana w artykule! Co za przytłaczający sukces!